In [2]:
import pandas as pd
import json
import numpy
from pandas.io.json import json_normalize
from scipy.spatial.distance import cosine
import csv
from pyspark import SparkContext
from pyspark import SparkConf
from pyspark.sql import SQLContext
from pyspark.sql.functions import udf #user defined function
from pyspark.sql.functions import lit, col
from pyspark.sql.types import *
from pyspark.sql import HiveContext
import ast

In [3]:
sc = SparkContext("local", "Region Network")

In [4]:
# loading the raw data
df = pd.read_csv('../../data/CDR/hash/sample.csv') 
df.columns = ['index','time','source','dest','call']
sqlCtx = SQLContext(sc)
cs_df = sqlCtx.createDataFrame(df)

In [5]:
df.head()

,index,time,source,dest,call
0,11340721,1383327600000,7853,1005,0.000323
1,11340722,1383327600000,7853,1012,0.000116
2,11340723,1383327600000,7853,1103,0.000576
3,11340724,1383327600000,7853,1117,0.000427
4,11340725,1383292200000,7853,1131,0.000671


In [6]:

# loading the region-cell data
table = pd.read_csv('../../data/CDR/hash/intersect.csv', header = None) 
table.columns = ['region', 'proportions']
table.index = table.region
table.sort_values(['region'], inplace=True)

# loading the cell-proportion data
prop_table = pd.read_csv('../../data/CDR/hash/cell_intersect.csv', header = None) 
prop_table.columns = ['cell', 'proportions']
prop_table.index = prop_table.cell
prop_table.sort_values(['cell'], inplace=True)


In [18]:
table.head()

,region,proportions
region,,
1,1,"{5255: 0.012533869596943227, 5256: 0.810203385..."
2,2,"{5760: 1.0, 5761: 0.8881973502712377, 5762: 0...."
3,3,"{5761: 0.10963649144088197, 5762: 0.0166565829..."
4,4,"{6656: 0.08041696535588132, 6657: 0.1769243460..."
5,5,"{5761: 0.0021661582885276975, 5762: 0.51612498..."


In [ ]:


def get_cells_per_region(table, region_id):
    ids = table.iloc[region_id].proportions
    ids = ast.literal_eval(table.get_value(region_id, "proportions"))
    return ids.keys()

def get_call_data(source, dest):        
    source_dict = get_cells_per_region(table, source)
    dest_dict = get_cells_per_region(table, dest)
    query = "SELECT * from cs_df WHERE "
    
    cs_df.registerTempTable("cs_df")
    i = 1    
    
    for skey in source_dict:                  
        query += "source = " + str(skey) 
        if len(source_dict) > i:
            query += " OR "
        i += 1
    
    subset = sqlCtx.sql(query)
    subset.registerTempTable("subset")
    
    i=1
    query = "SELECT * from subset WHERE "
            
    for dkey in dest_dict:  
        query += "dest = " + str(dkey)
        if len(dest_dict) > i:
            query += " OR "
        i += 1
        
    subset2 = sqlCtx.sql(query)
            
    return subset2

def calculate_actual_call(s_cell, d_cell, call, s_region, d_region):
    """
        Create another column on the subset DataFrame that is proportional to the regions.
    """
    source_prop = ast.literal_eval(prop_table.get_value(s_cell, "proportions"))
    dest_prop = ast.literal_eval(prop_table.get_value(d_cell, "proportions"))

    try:
        final = source_prop[str(s_region)] * dest_prop[str(d_region)] * call
    except:
        final = 0
    
    return final

In [ ]:

schema = StructType([
            StructField("time", IntegerType(), nullable=False),
            StructField("source_region", IntegerType(), nullable=False),
            StructField("dest_region", IntegerType(), nullable=False),
            StructField("adjusted_call", FloatType(), nullable=False)
    ])

region_network = sqlCtx.createDataFrame(sc.emptyRDD(), schema)
udf_calls = udf(calculate_actual_call, FloatType())

for s in range(1,81):
    for d in range(1,81):
        # get a subset of records for the source and dest
        subdf = get_call_data(s, d)        
        subdf = subdf.withColumn("source_region", lit(s))
        subdf = subdf.withColumn("dest_region", lit(d))
        print (s, d)
        # create a column with adjusted call values
        try:
            subdf = subdf.withColumn("adjusted_call", calculate_actual_call("source", "dest", "call", "source_region", "dest_region"))
        except:
            continue
        
        # do aggregation for 
        subdf.groupBy("time").agg({"adjusted_call": "sum"})
        region_network = region_network.unionAll(subdf)
        
region_network.show()
region_network.toPandas().to_csv('../../CDR/generated/region-network.csv')